In [1]:
import gurobipy as gp
import numpy as np
import pandas as pd

In [2]:
csv_data = pd.read_csv('../data/Final_Energy_dataset.csv')
csv_data.drop(columns=['Date','emissions'], inplace=True)
csv_data.set_index(pd.RangeIndex(0,52608),inplace=True)
csv_data.fillna(0, inplace=True)
# csv_data = (csv_data.loc[35088:52607]).set_index(pd.RangeIndex(0,17520))
# csv_data = (csv_data.loc[17520:35087]).set_index(pd.RangeIndex(0,17568))
csv_data = (csv_data.loc[0:17519]).set_index(pd.RangeIndex(0,17520))

In [ ]:
model = gp.Model('bess_opti')

In [4]:
timeslot = range(0, 17520, 1)

In [5]:
load = csv_data['load_1'].values
pv = csv_data['pv_1'].values
price = csv_data['price'].values

In [ ]:
action = model.addVars(timeslot, lb=-2.3, ub=2.3, name="action")
soc = model.addVars(timeslot, lb=0, ub=13.5, name="soc")

# Add constraints
for t in range(17520):
    # Battery State of Charge (SOC) dynamics
    if t == 0:
        model.addConstr(soc[t] == 0 + action[t], name=f"soc_initial_{t}")
    else:
        model.addConstr(soc[t] == soc[t-1] + action[t], name=f"soc_dynamics_{t}")

# Define net load and cost
net_load = {t: load[t] - pv[t] + action[t] for t in range(17520)}

# Objective: Minimize the cost
cost = gp.quicksum(price[t] * net_load[t] for t in range(17520))
model.setObjective(cost, gp.GRB.MINIMIZE)
model.optimize()

# write solution in dataframe
solution = pd.DataFrame(index=timeslot)
solution['action'] = [action[t].x for t in timeslot]
solution['soc'] = [soc[t].x for t in timeslot]
solution['soc'].plot()
# for v in model.getVars():
    # print(f"{v.varName}: {v.x}")